In [12]:
import cellshape_helper as helper
import trimesh
from cellshape_helper.vendor.pytorch_geometric_files import (
    read_off,
    sample_points,
)
from pyntcloud import PyntCloud
import pandas as pd
from tifffile import imread
import trimesh
from skimage.measure import marching_cubes
from tqdm import tqdm
from cellshape_helper.util import create_dir_if_not_exist
from pathlib import Path


PATH_TO_TIF_FILES = (
    "/home/mvries/Documents/Datasets/MedMNIST/vesselmnist3d/test/Aneurysm/"
)
PATH_TO_SAVE_MESH = "/home/mvries/Documents/Datasets/MedMNIST/vesselmnist3d/test/Aneurysm_mesh/"
PATH_TO_SAVE_MESH_smooth = "/home/mvries/Documents/Datasets/MedMNIST/vesselmnist3d/test/Aneurysm_mesh_smoothed_4096/"
PATH_TO_SAVE_PC = "/home/mvries/Documents/Datasets/MedMNIST/vesselmnist3d/test/Aneurysm_pointcloud_smoothed/"
NUM_POINTS = 2048


def mesh_to_pc(mesh_directory, smoothed_dir, num_points, save_dir):
    p = Path(mesh_directory)
    files = list(p.glob("*.off"))
    failed_files = []
    for mesh_file in tqdm(files):
        try:
            mesh_file_path = Path(mesh_file)

            mesh = trimesh.load(mesh_file_path)
            smoothed = trimesh.smoothing.filter_laplacian(mesh)

            create_dir_if_not_exist(smoothed_dir)
            smoothed.export(smoothed_dir + mesh_file_path.name)

            data = read_off(smoothed_dir + mesh_file_path.name)
            # changed to .numpy() to avoid issue with pyntcloud
            points = sample_points(data=data, num=num_points).numpy()
            save_to_points_path = save_dir
            create_dir_if_not_exist(save_to_points_path)
            # split_string = mesh_file_path.name.split(".")
            # file_name = split_string[0]
            file_name = mesh_file_path.name[:-4]
            cloud = PyntCloud(
                pd.DataFrame(data=points, columns=["x", "y", "z"])
            )
            cloud.to_file(save_to_points_path + file_name + ".ply")
        except Exception as e:
            print(e)
            failed_files.append(mesh_file.name)
            continue

    return failed_files

In [13]:
mesh_to_pc(PATH_TO_SAVE_MESH, PATH_TO_SAVE_MESH_smooth, 2048, PATH_TO_SAVE_PC)

100%|███████████████████████████████████████████| 43/43 [00:01<00:00, 23.07it/s]


[]

In [15]:
PATH_TO_SAVE_MESH = (
    "/home/mvries/Documents/Datasets/MedMNIST/vesselmnist3d/test/Healthy_mesh/"
)
PATH_TO_SAVE_MESH_smooth = "/home/mvries/Documents/Datasets/MedMNIST/vesselmnist3d/test/Healthy_mesh_smoothed/"
PATH_TO_SAVE_PC = "/home/mvries/Documents/Datasets/MedMNIST/vesselmnist3d/test/Healthy_pointcloud_smoothed/"
NUM_POINTS = 2048

mesh_to_pc(PATH_TO_SAVE_MESH, PATH_TO_SAVE_MESH_smooth, 2048, PATH_TO_SAVE_PC)

100%|█████████████████████████████████████████| 339/339 [00:12<00:00, 27.23it/s]


[]

In [16]:
PATH_TO_SAVE_MESH = "/home/mvries/Documents/Datasets/MedMNIST/vesselmnist3d/train/Aneurysm_mesh/"
PATH_TO_SAVE_MESH_smooth = "/home/mvries/Documents/Datasets/MedMNIST/vesselmnist3d/train/Aneurysm_mesh_smoothed/"
PATH_TO_SAVE_PC = "/home/mvries/Documents/Datasets/MedMNIST/vesselmnist3d/train/Aneurysm_pointcloud_smoothed/"
NUM_POINTS = 2048
mesh_to_pc(PATH_TO_SAVE_MESH, PATH_TO_SAVE_MESH_smooth, 2048, PATH_TO_SAVE_PC)

100%|█████████████████████████████████████████| 150/150 [00:06<00:00, 22.68it/s]


[]

In [17]:
PATH_TO_SAVE_MESH = "/home/mvries/Documents/Datasets/MedMNIST/vesselmnist3d/train/Healthy_mesh/"
PATH_TO_SAVE_MESH_smooth = "/home/mvries/Documents/Datasets/MedMNIST/vesselmnist3d/train/Healthy_mesh_smoothed/"
PATH_TO_SAVE_PC = "/home/mvries/Documents/Datasets/MedMNIST/vesselmnist3d/train/Healthy_pointcloud_smoothed/"
NUM_POINTS = 2048

mesh_to_pc(PATH_TO_SAVE_MESH, PATH_TO_SAVE_MESH_smooth, 2048, PATH_TO_SAVE_PC)

  4%|█▊                                       | 51/1185 [00:01<00:47, 23.93it/s]/home/mvries/anaconda3/envs/dcc/lib/python3.8/site-packages/trimesh/smoothing.py:83: RuntimeWarning: invalid value encountered in double_scalars
  vertices *= ((vol_ini / vol_new) ** (1.0 / 3.0))
  5%|██                                       | 61/1185 [00:02<00:34, 32.92it/s]

invalid multinomial distribution (encountering probability entry < 0)


100%|███████████████████████████████████████| 1185/1185 [00:38<00:00, 30.42it/s]


['00888.off']

In [42]:
path = (
    "/home/mvries/Documents/Datasets/OPM/SingleCellFromNathan_17122021/Plate2/"
)

PATH_TO_TIF_FILES = path + "stacked/"
PATH_TO_SAVE_MESH = path + "stacked_off/raw/"
PATH_TO_SAVE_PC = path + "stacked_pointcloud/"
NUM_POINTS = 2048


def mesh_to_pc(mesh_directory, smoothed_dir, num_points, save_dir):
    p = Path(mesh_directory)
    files = list(p.glob("**/*.off"))
    failed_files = []
    for mesh_file in tqdm(files[27687:]):
        try:
            treatment = str(mesh_file.parent).replace(str(mesh_directory), "")

            mesh_file_path = Path(mesh_file)

            mesh = trimesh.load(mesh_file_path)
            smoothed = trimesh.smoothing.filter_laplacian(mesh)

            create_dir_if_not_exist(smoothed_dir + treatment + "/")
            smoothed.export(
                smoothed_dir + treatment + "/" + mesh_file_path.name
            )

            data = read_off(
                smoothed_dir + treatment + "/" + mesh_file_path.name
            )
            # changed to .numpy() to avoid issue with pyntcloud
            points = sample_points(data=data, num=num_points).numpy()
            save_to_points_path = save_dir
            create_dir_if_not_exist(save_to_points_path + treatment + "/")
            # split_string = mesh_file_path.name.split(".")
            # file_name = split_string[0]
            file_name = mesh_file_path.name[:-4]
            cloud = PyntCloud(
                pd.DataFrame(data=points, columns=["x", "y", "z"])
            )
            cloud.to_file(
                save_to_points_path + treatment + "/" + file_name + ".ply"
            )
        except Exception as e:
            print(e)
            failed_files.append(mesh_file.name)
            continue

    return failed_files


# mesh_to_pc(PATH_TO_SAVE_MESH, path + "stacked_off_smoothed/" , 2048, path + "stacked_pointcloud_smoothed/")

In [43]:
path = (
    "/home/mvries/Documents/Datasets/OPM/SingleCellFromNathan_17122021/Plate1/"
)

PATH_TO_TIF_FILES = path + "stacked/"
PATH_TO_SAVE_MESH = path + "stacked_off/raw/"
PATH_TO_SAVE_PC = path + "stacked_pointcloud/"
NUM_POINTS = 2048

mesh_to_pc(
    PATH_TO_SAVE_MESH,
    path + "stacked_off_smoothed/",
    2048,
    path + "stacked_pointcloud_smoothed/",
)

 18%|█▊        | 898/4910 [03:20<11:59,  5.58it/s]/home/mvries/anaconda3/envs/dcc/lib/python3.8/site-packages/trimesh/smoothing.py:83: RuntimeWarning: invalid value encountered in double_scalars
  vertices *= ((vol_ini / vol_new) ** (1.0 / 3.0))
 18%|█▊        | 901/4910 [03:20<09:05,  7.35it/s]

invalid multinomial distribution (encountering probability entry < 0)


 21%|██        | 1035/4910 [03:51<18:31,  3.49it/s]/home/mvries/anaconda3/envs/dcc/lib/python3.8/site-packages/trimesh/smoothing.py:83: RuntimeWarning: invalid value encountered in double_scalars
  vertices *= ((vol_ini / vol_new) ** (1.0 / 3.0))
 21%|██        | 1038/4910 [03:52<10:50,  5.96it/s]

invalid multinomial distribution (encountering probability entry < 0)


 25%|██▌       | 1248/4910 [04:42<17:06,  3.57it/s]/home/mvries/anaconda3/envs/dcc/lib/python3.8/site-packages/trimesh/smoothing.py:83: RuntimeWarning: invalid value encountered in double_scalars
  vertices *= ((vol_ini / vol_new) ** (1.0 / 3.0))


...

invalid multinomial distribution (encountering probability entry < 0)


 99%|█████████▉| 4878/4910 [17:15<00:06,  4.91it/s]/home/mvries/anaconda3/envs/dcc/lib/python3.8/site-packages/trimesh/smoothing.py:83: RuntimeWarning: invalid value encountered in double_scalars
  vertices *= ((vol_ini / vol_new) ** (1.0 / 3.0))


invalid multinomial distribution (encountering probability entry < 0)


100%|██████████| 4910/4910 [17:22<00:00,  4.71it/s]


['0054_0050_accelerator_20210315_bakal01_erk_main_21-03-15_12-37-27.off',
 '0128_0178_accelerator_20210315_bakal01_erk_main_21-03-15_12-37-27.off',
 '0054_0145_accelerator_20210315_bakal01_erk_main_21-03-15_12-37-27.off',
 '0129_0217_accelerator_20210315_bakal01_erk_main_21-03-15_12-37-27.off',
 '0128_0205_accelerator_20210315_bakal01_erk_main_21-03-15_12-37-27.off',
 '0140_0093_accelerator_20210315_bakal01_erk_main_21-03-15_12-37-27.off',
 '0021_0239_accelerator_20210315_bakal01_erk_main_21-03-15_12-37-27.off',
 '0140_0143_accelerator_20210315_bakal01_erk_main_21-03-15_12-37-27.off',
 '0042_0235_accelerator_20210315_bakal01_erk_main_21-03-15_12-37-27.off',
 '0042_0200_accelerator_20210315_bakal01_erk_main_21-03-15_12-37-27.off',
 '0042_0192_accelerator_20210315_bakal01_erk_main_21-03-15_12-37-27.off',
 '0100_0225_accelerator_20210315_bakal01_erk_main_21-03-15_12-37-27.off',
 '0080_0120_accelerator_20210315_bakal01_erk_main_21-03-15_12-37-27.off',
 '0019_0121_accelerator_20210315_bakal

In [2]:
import cellshape_helper as helper


PATH_TO_TIF_FILES = (
    "/home/mvries/Documents/Datasets/ShapeAE_datasets/RedBloodCell/obj/"
)
PATH_TO_SAVE_MESH = (
    "/home/mvries/Documents/Datasets/ShapeAE_datasets/RedBloodCell/mesh/"
)
PATH_TO_SAVE_PC = "/home/mvries/Documents/Datasets/ShapeAE_datasets/RedBloodCell/pointcloud_4096/"
NUM_POINTS = 4096

helper.tif_to_pc_directory(
    PATH_TO_TIF_FILES, PATH_TO_SAVE_MESH, PATH_TO_SAVE_PC, 4096
)

100%|██████████| 825/825 [01:34<00:00,  8.72it/s]
